Setup

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import torch
import CNN

# from config_plots import global_settings
# global_settings()
import warnings
warnings.filterwarnings("ignore")

Hyper parameters

In [2]:
# defining global variables
experimentName = "test_22987_suffix50_11"
showListOfSpecies = False

from configParser import ConfigParser, getModelName
config_parser = ConfigParser(experimentName)
    
experimentName = experimentName + "/multi-trial"
import TrialStatistics
ts = TrialStatistics.TrialStatistics(experimentName)

Use_old_dataLoader = False
if Use_old_dataLoader:
    import dataLoader
else:
    import dataLoader2 as dataLoader

Cuda support

In [3]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


Iterate

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from confusion_matrix_plotter import plot_confusion_matrix2, generate_classification_report
import numpy as np
import os
import progressbar

paramsIterator = config_parser.getHyperpIter()  
number_of_experiments = sum(1 for e in paramsIterator)
paramsIterator = config_parser.getHyperpIter()  
experiment_index = 0

with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
    for experiment_params in paramsIterator:
        bar.update(experiment_index)
        experiment_params["numOfTrials"] = experiment_params["numOfTrials"] or 1

        print("experiment ", experiment_index, "/", number_of_experiments, ": ", experiment_params)

        # load images
        dataset = dataLoader.FishDataset(experiment_params, showListOfSpecies)
        speciesList = dataset.getSpeciesList()
        numberOfSpecies = len(speciesList)

        confusionMatricesPerTrial = []

        for i in range(experiment_params["numOfTrials"]):
            trialName = experimentName+"/"+getModelName(experiment_params, i)

            # Train/Load model
            model = CNN.CNN(numberOfSpecies, experiment_params)
            if os.path.exists(CNN.getModelFile(trialName)):
                _, _, epochs, time_elapsed = CNN.loadModel(model, trialName)
                test_loader = dataLoader.loadTestLoader(trialName)
                print("Model {0} loaded!".format(trialName))
            else:
                train_loader, validation_loader, test_loader = dataLoader.getLoadersFromDataset(dataset, experiment_params, trialName)
                _, _, epochs, time_elapsed = CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName)

            # Update trial outcomes for statistics
            predlist, lbllist = CNN.getLoaderPredictions(test_loader, model)
            ts.addTrialPredictions(experiment_params, predlist, lbllist, numberOfSpecies)

            ts.addTrial(experiment_params,
                    {'loss': CNN.getCrossEntropyFromLoader(test_loader, model),
                     'accuracy': CNN.getAccuracyFromLoader(test_loader, model),
                     'time': time_elapsed,
                     'epochs': epochs
                    }, i)
        
        experiment_index = experiment_index + 1

N/A% (0 of 24) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

experiment  0 / 24 :  {'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 200, 'n_epochs': 1000, 'numOfTrials': 1, 'kernels': [16, 32], 'kernelSize': 5, 'patience': 50, 'imageDimension': 28, 'n_channels': 1, 'useZCAWhitening': False}
Loading dataset...
Loading saved dataset structure...


100% (550 of 550) |######################| Elapsed Time: 0:00:05 Time:  0:00:05


Model test_22987_suffix50_11/multi-trial/tc0_vc0_bs200_e1000_ks5_k16-32_p50_d28_c1_zcaFalse_id0 loaded!


  4% (1 of 24) |#                        | Elapsed Time: 0:00:49 ETA:   0:18:49

...

100% (550 of 550) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


Model test_22987_suffix50_11/multi-trial/tc0_vc0_bs200_e1000_ks5_k16-32_p50_d236_c1_zcaFalse_id0 loaded!


 70% (17 of 24) |#################       | Elapsed Time: 0:11:47 ETA:   0:02:21

experiment  17 / 24 :  {'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 200, 'n_epochs': 1000, 'numOfTrials': 1, 'kernels': [16, 32], 'kernelSize': 5, 'patience': 50, 'imageDimension': 236, 'n_channels': 1, 'useZCAWhitening': True}
Loading dataset...
Loading saved dataset structure...


100% (550 of 550) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


Analyze

In [ ]:
ts.showStatistics()
ts.showStatistics(False)
ts.saveStatistics()
ts.saveStatistics(False)

paramsIterator = config_parser.getHyperpIter() 
for experiment_params in paramsIterator:
    experiment_params["numOfTrials"] = experiment_params["numOfTrials"] or 1
    
    print("experiment: ", experiment_params)
    
    dataset = dataLoader.FishDataset(experiment_params, showListOfSpecies)
    speciesList = dataset.getSpeciesList()
    ts.printTrialConfusionMatrix(experiment_params, speciesList, True)